# Resumidor de notícias - Resume-AI

## Descrição e Motivação do Problema

#### Problema: Resumir automaticamente notícias em português de forma eficiente e precisa.
#### Motivação: O volume de notícias cresce exponencialmente, tornando difícil acompanhar informações relevantes. Um sistema de resumo automático ajuda a poupar tempo e identificar conteúdos importantes rapidamente.
#### Solução que visa gerar material para conteúdos curtos nas principais redes sociais atuais (TikTok, Shorts do Youtube, Stories do Instagram), rentabibilizar projeto com assinatura de sistema baseado neste código.

In [53]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
from rouge_score import rouge_scorer
import requests
from bs4 import BeautifulSoup

# Carregar modelo maior e tokenizador
modelo = T5ForConditionalGeneration.from_pretrained("t5-large")
tokenizador = T5Tokenizer.from_pretrained("t5-large")

# Função para fazer scraping do artigo
def web_scraping_artigo(urls):
    texto_artigo = ""
    for url in urls:
        resposta = requests.get(url)
        soup = BeautifulSoup(resposta.text, 'html.parser')
        paragrafos = soup.find_all('p')
        texto_artigo += " ".join([p.get_text() for p in paragrafos])
    return texto_artigo

def resumir_texto(texto):
    texto_entrada = "summarize: " + texto
    ids_entrada = tokenizador.encode(texto_entrada, return_tensors="pt", max_length=512, truncation=True, padding="longest")

    ids_resumo = modelo.generate(
        ids_entrada,
        max_length=500,
        num_beams=8,
        length_penalty=5,
        early_stopping=True
    )

    resumo = tokenizador.decode(ids_resumo[0], skip_special_tokens=True)
    return resumo

def avaliar_resumo(resumo_referencia, resumo_gerado):
    avaliador = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    pontuacoes = avaliador.score(resumo_referencia, resumo_gerado)
    return pontuacoes


def processar_multiplas_urls(urls, resumos_referencia):
    todos_resultados = []

    for idx, url in enumerate(urls):
        texto_artigo = web_scraping_artigo([url])
        resumo_gerado = resumir_texto(texto_artigo)

        resumo_referencia = resumos_referencia[idx]

        resultados_avaliacao = avaliar_resumo(resumo_referencia, resumo_gerado)
        todos_resultados.append({
            "url": url,
            "resumo_gerado": resumo_gerado,
            "resultados_avaliacao": resultados_avaliacao
        })

    return todos_resultados

# URLs utilizadas
urls = [
    'https://g1.globo.com/carros/noticia/2024/11/30/veja-a-lista-de-10-carros-que-reprovaram-no-teste-de-seguranca-do-latin-ncap.ghtml',
    'https://oglobo.globo.com/mundo/noticia/2024/11/30/tres-dias-apos-cessar-fogo-israel-ataca-hezbollah-no-sul-do-libano.ghtml',
    'https://g1.globo.com/ms/mato-grosso-do-sul/noticia/2024/11/29/principal-vitima-de-venda-de-sentencas-bb-teve-prejuizos-que-ultrapassam-r200-milhoes.ghtml',
    'https://ge.globo.com/futebol/times/vasco/noticia/2024/11/30/torcedor-do-vasco-pedala-do-espirito-santo-ao-rio-para-assistir-ao-jogo-contra-o-atletico-go.ghtml',
    'https://oglobo.globo.com/cultura/noticia/2024/11/30/empresario-que-pagou-r-37-milhoes-por-banana-colada-com-fita-se-oferece-para-comprar-r-150-mil-em-banca-que-vendeu-fruta-original.ghtml'
]

# Resumos de referência
resumos_referencia = [
    'A Mitsubishi L200 Triton foi a primeira picape a alcançar a nota máxima de cinco estrelas nos testes de segurança do Latin NCAP, seguindo os novos protocolos implementados desde 2020. Este resultado é uma grande melhoria, considerando que a versão de 2019 do veículo havia obtido zero estrelas.',
    'Três dias após o cessar-fogo, Israel atacou posições do Hezbollah no sul do Líbano, mantendo a tensão elevada na região. A ofensiva ocorre enquanto o país se mantém alerta sobre a possibilidade de violação do acordo de cessar-fogo, ao mesmo tempo em que intensifica os ataques em Gaza.',
    'O Banco do Brasil é uma das principais vítimas de um esquema de venda de decisões judiciais, que está sendo investigado pela Polícia Federal. O caso envolve desembargadores do Tribunal de Justiça de Mato Grosso do Sul e tem causado prejuízos que ultrapassam os R$ 200 milhões.',
    'João Willians, torcedor do Vasco, fez uma grande jornada ao pedalar de Cariacica (ES) até o Rio de Janeiro para assistir ao jogo do time contra o Atlético-GO. Além disso, o jogador Paulinho foi novamente relacionado para o time após um longo período de ausência.',
    'O empresário chinês de criptomoedas Justin Sun ficou famoso após pagar R$ 37 milhões por uma "banana colada com fita". Agora, ele anunciou que planeja comprar 100 mil bananas, equivalentes a R$ 150 mil, do quiosque em Manhattan onde a banana original foi vendida, gerando mais repercussão nas redes sociais.'
]

# Processar as URLs e avaliar os resumos
resultados = processar_multiplas_urls(urls, resumos_referencia)

# Exibir os resultados
for resultado in resultados:
    print(f"URL: {resultado['url']}")
    print(f"Resumo gerado: {resultado['resumo_gerado']}")
    print(f"Avaliação ROUGE: {resultado['resultados_avaliacao']}\n")


URL: https://g1.globo.com/carros/noticia/2024/11/30/veja-a-lista-de-10-carros-que-reprovaram-no-teste-de-seguranca-do-latin-ncap.ghtml
Resumo gerado: a Mitsubishi L200 triton foi a primeira picape a conseguir cinco estrelas conforme os ltimos protocolos de avaliaço do Latin NCAP, em vigor desde 2020 . o resultado desta semana acontece depois da verso de 2019 do veculo ter recebido nota zero .
Avaliação ROUGE: {'rouge1': Score(precision=0.6190476190476191, recall=0.5, fmeasure=0.5531914893617021), 'rouge2': Score(precision=0.34146341463414637, recall=0.27450980392156865, fmeasure=0.30434782608695654), 'rougeL': Score(precision=0.5238095238095238, recall=0.4230769230769231, fmeasure=0.4680851063829788)}

URL: https://oglobo.globo.com/mundo/noticia/2024/11/30/tres-dias-apos-cessar-fogo-israel-ataca-hezbollah-no-sul-do-libano.ghtml
Resumo gerado: exército israelense atacou neste sábado posiçes do Hezbollah no Sul do Lbano, mantendo o pas em alerta sobre um cessar-fogo frágil, assinado há t

# Ranking Atualizado (Melhores Resumos)

| **Posição** | **Resumo (URL)**                                     | **Média F1** |
|-------------|------------------------------------------------------|--------------|
| **1º**      | [Banana colada com fita](https://oglobo.globo.com/cultura/noticia/2024/11/30/empresario-que-pagou-r-37-milhoes-por-banana-colada-com-fita-se-oferece-para-comprar-r-150-mil-em-banca-que-vendeu-fruta-original.ghtml) | **0.553**    |
| **2º**      | [Teste do Latin NCAP](https://g1.globo.com/carros/noticia/2024/11/30/veja-a-lista-de-10-carros-que-reprovaram-no-teste-de-seguranca-do-latin-ncap.ghtml)                                              | **0.553**    |
| **3º**      | [Torcedor do Vasco](https://ge.globo.com/futebol/times/vasco/noticia/2024/11/30/torcedor-do-vasco-pedala-do-espirito-santo-ao-rio-para-assistir-ao-jogo-contra-o-atletico-go.ghtml)                           | **0.453**    |
| **4º**      | [Esquema no TJMS](https://g1.globo.com/ms/mato-grosso-do-sul/noticia/2024/11/29/principal-vitima-de-venda-de-sentencas-bb-teve-prejuizos-que-ultrapassam-r200-milhoes.ghtml)                             | **0.398**    |
| **5º**      | [Ataques de Israel](https://oglobo.globo.com/mundo/noticia/2024/11/30/tres-dias-apos-cessar-fogo-israel-ataca-hezbollah-no-sul-do-libano.ghtml)                                                       | **0.325**    |

---

## Análise

### Melhor Resumo
Os resumos da **"Banana colada com fita"** e do **"Teste do Latin NCAP"** empataram com a maior média F1 (**0.553**). Ambos apresentaram alta fidelidade e abrangência em relação aos textos de referência.

### Pior Resumo
O resumo dos **"Ataques de Israel"** apresenta a menor média F1 (**0.325**), evidenciando menor fidelidade e dificuldade em abordar a complexidade do conteúdo original.